In [1]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from matplotlib.colors import Normalize
from itertools import combinations
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFE
from bayes_opt import BayesianOptimization
from sklearn.inspection import PartialDependenceDisplay

In [2]:
def run_bayesian_optimization(X_train_all, y_train_all):
    def black_box_function(learning_rate, n_estimators, min_samples_split, max_features, max_depth, max_leaf_nodes):
        params = {
            'learning_rate': max(learning_rate, 1e-3),
            'n_estimators': int(n_estimators),
            'min_samples_split': int(min_samples_split),
            'max_features': min(max_features, 0.999),
            'max_depth': int(max_depth),
            'max_leaf_nodes': int(max_leaf_nodes),
            'random_state': 2
        }
        
        model = GradientBoostingRegressor(**params)
        loo = LeaveOneOut()
        preds, truths = [], []
        
        for train_idx, val_idx in loo.split(X_train_all):
            X_train, X_val = X_train_all[train_idx], X_train_all[val_idx]
            y_train, y_val = y_train_all[train_idx], y_train_all[val_idx]
            model.fit(X_train, y_train.ravel())
            preds.append(model.predict(X_val)[0])
            truths.append(y_val[0])
            
        return r2_score(truths, preds)

    pbounds = {
        'learning_rate': (0.001, 0.2),
        'n_estimators': (10, 500),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 1.0),
        'max_depth': (1, 5),
        'max_leaf_nodes': (2, 15)
    }

    optimizer = BayesianOptimization(
        f=black_box_function,
        pbounds=pbounds,
        random_state=1
    )
    optimizer.maximize(init_points=15, n_iter=20)
    return optimizer.max['params']

In [4]:
data = pd.read_excel(r"C:\Users\HP\Desktop\Data.xlsx",
                    sheet_name='16+3',
                    index_col=0,
                    header=0)
features = data[['lg(O3)', 'lg(H2O2)', 'pH', 'TOC', 'FMax2']]
data1 = features.iloc[0:18]

df_GAN_origin_all = pd.read_excel(
    r"C:\Users\HP\jupyternotebook\MLofCC\GAN\GAN800.xlsx",
    header=0)
df_GAN_origin = df_GAN_origin_all[['lg(O3)', 'lg(H2O2)', 'pH', 'TOC']]

scaler1 = MinMaxScaler()
feature_columns = ['lg(O3)', 'lg(H2O2)', 'pH', 'TOC','FMax2']
scaled_features = scaler1.fit_transform(data1[feature_columns])
data3 = pd.DataFrame(scaled_features, columns=feature_columns)


X = data3[['lg(O3)', 'lg(H2O2)', 'pH', 'FMax2']].values
y = data3['TOC'].values

def black_box_function(learning_rate, n_estimators, min_samples_split, max_features, max_depth, max_leaf_nodes):
    model = GradientBoostingRegressor(
            learning_rate=learning_rate,
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999),
            max_depth=int(max_depth),
            max_leaf_nodes=int(max_leaf_nodes),
            random_state=2
        )
    loo = LeaveOneOut()
    preds, truths = [], []
    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train)
        preds.append(model.predict(X_test)[0])
        truths.append(y_test[0])
    return r2_score(truths, preds)

optimizer = BayesianOptimization(
        f=black_box_function,
        pbounds={
            'learning_rate': (0.001, 0.2),
            'n_estimators': (10, 500),
            'min_samples_split': (2, 25),
            'max_features': (1, 4),
            'max_depth': (1, 5),
            'max_leaf_nodes': (2, 15)
        },
        random_state=42
    )
optimizer.maximize(init_points=15, n_iter=25)
best_params = optimizer.max['params']
    
model = GradientBoostingRegressor(
        learning_rate=best_params['learning_rate'],
        n_estimators=int(best_params['n_estimators']),
        max_leaf_nodes=int(best_params['max_leaf_nodes']),
        max_features=min(best_params['max_features'], 0.999),
        min_samples_split=int(best_params['min_samples_split']),
        max_depth=int(best_params['max_depth']),
        random_state=2
    )
model.fit(X, y)

model_FMax2 = LinearRegression()
model_FMax2.fit(data3[['lg(O3)']], data3['FMax2'])
    
scaler2 = MinMaxScaler()
train_features = data1[['lg(O3)', 'lg(H2O2)', 'pH', 'TOC']].values  
scaler2.fit(train_features)
gan_features = scaler2.transform(df_GAN_origin[['lg(O3)', 'lg(H2O2)', 'pH', 'TOC']])
df_GAN = pd.DataFrame(gan_features, columns=['lg(O3)', 'lg(H2O2)', 'pH', 'TOC'])
    
df_GAN['FMax2'] = model_FMax2.predict(df_GAN[['lg(O3)']])

X_new = df_GAN[['lg(O3)', 'lg(H2O2)', 'pH', 'FMax2']]
y_pred_new = model.predict(X_new)

toc_min = scaler2.data_min_[-1]
toc_max = scaler2.data_max_[-1]
y_pred_orig = y_pred_new * (toc_max - toc_min) + toc_min

|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.7892    | 0.07553   | 4.803     | 3.196     | 9.783     | 5.588     | 86.44     |
| 2         | 0.7396    | 0.01256   | 4.465     | 2.803     | 11.2      | 2.473     | 485.3     |
| 3         | 0.6762    | 0.1667    | 1.849     | 1.545     | 4.384     | 8.998     | 267.1     |
| 4         | 0.8168    | 0.08696   | 2.165     | 2.836     | 3.813     | 8.719     | 189.5     |
| 5         | 0.6939    | 0.09176   | 4.141     | 1.599     | 8.685     | 15.63     | 32.76     |
| 6         | -0.1211   | 0.1219    | 1.682     | 1.195     | 14.34     | 24.21     | 406.1     |
| 7         | 0.7073    | 0.06162   | 1.391     | 3.053     | 7.722     | 4.807     | 252.6     |
| 8         | 0.738     | 0.007843  | 4.637     | 1.776     | 10.61     | 9.169     | 264.8     |
| 9         | -0.121

E:\ANACONDA\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
E:\ANACONDA\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [16]:
# ================== 0.05，36 ==================
all_train_true = []
all_train_pred = []
all_test_true = []
all_test_pred = []

r2_results = []
rmse_results = []

scaler3 = MinMaxScaler()
feature_columns3 = ['lg(O3)', 'lg(H2O2)', 'pH']
scaler3.fit(data1[feature_columns3])
# ================== Training cycle ==================
for iter in range(10):
    print(f"\n========== Iteration times {iter+1}/10 ==========")
    
    valid_indices = []
    for i in range(len(y_pred_orig)):
        if abs(df_GAN_origin.iloc[i]['TOC'] - y_pred_orig[i]) < 0.05:
            valid_indices.append(i)
    np.random.seed(iter)  

    if len(valid_indices) == 0:
        raise ValueError("No valid samples were found, please adjust the screening threshold.")

    try:
        selected_indices = np.random.choice(valid_indices, size=36, replace=False)
    except ValueError:
        selected_indices = np.random.choice(valid_indices, size=36, replace=True)

    selected_data = df_GAN_origin.iloc[selected_indices]


    test_indices = np.random.choice(data1.index, size=4, replace=False)
    test_set = data1.loc[test_indices]
    train_set = data1.drop(test_indices)
    

    combined_data = pd.concat([train_set, selected_data])
    X_final = scaler3.transform(combined_data[['lg(O3)', 'lg(H2O2)', 'pH']])
    y_final = combined_data['TOC'].values.reshape(-1, 1)

    
    best_params = run_bayesian_optimization(X_final, y_final)
    final_model = GradientBoostingRegressor(learning_rate=best_params['learning_rate'],
        n_estimators=int(best_params['n_estimators']),
        min_samples_split=int(best_params['min_samples_split']),
        max_features=best_params['max_features'],
        max_depth=int(best_params['max_depth']),
        max_leaf_nodes=int(best_params['max_leaf_nodes']),
        random_state=2)
    final_model.fit(X_final, y_final.ravel())

    train_pred = final_model.predict(X_final)
    all_train_true.extend(y_final.ravel().tolist())
    all_train_pred.extend(train_pred.tolist())

    X_test = scaler3.transform(test_set[['lg(O3)', 'lg(H2O2)', 'pH']])
    y_test = test_set['TOC'].values
    test_pred = final_model.predict(X_test)
    all_test_true.extend(y_test.tolist())
    all_test_pred.extend(test_pred.tolist())
    
    r2 = r2_score(y_test, test_pred)
    rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    
    r2_results.append(r2)
    rmse_results.append(rmse)
    print(f"Results of this round: R2={r2:.4f}, RMSE={rmse:.4f}")

print("\n================ Final result ================")
print(f"Average R2 score: {np.mean(r2_results):.4f} ± {np.std(r2_results):.4f}")
print(f"Average RMSE score: {np.mean(rmse_results):.4f} ± {np.std(rmse_results):.4f}")
print("\n Detailed R2 results:", [round(x, 4) for x in r2_results])
print("Detailed RMSE results:", [round(x, 4) for x in rmse_results])


========== 迭代次数 1/10 ==========
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.8465    | 0.08399   | 3.881     | 0.1001    | 5.93      | 5.375     | 55.25     |
| 2         | 0.8621    | 0.03807   | 2.382     | 0.4571    | 9.005     | 11.64     | 345.8     |
| 3         | 0.8487    | 0.04169   | 4.512     | 0.1246    | 10.72     | 11.6      | 283.8     |
| 4         | 0.7943    | 0.02894   | 1.792     | 0.8207    | 14.59     | 9.209     | 349.2     |
| 5         | 0.8145    | 0.1754    | 4.578     | 0.1765    | 2.508     | 5.906     | 440.3     |
| 6         | 0.8413    | 0.02057   | 2.684     | 0.9621    | 8.931     | 17.91     | 164.6     |
| 7         | 0.8155    | 0.1376    | 4.339     | 0.1165    | 11.75     | 24.74     | 376.6     |
| 8         | 0.8223    | 0.05681   | 4.157     | 0.1929    | 7.823     | 22.9      |

| 9         | 0.7768    | 0.05827   | 1.52      | 0.1174    | 10.82     | 6.867     | 140.1     |
| 10        | 0.7314    | 0.09882   | 1.213     | 0.6167    | 3.907     | 15.55     | 352.9     |
| 11        | 0.8631    | 0.02136   | 2.656     | 0.725     | 7.384     | 3.149     | 272.6     |
| 12        | 0.8614    | 0.1331    | 3.06      | 0.9501    | 9.625     | 22.78     | 77.36     |
| 13        | 0.8301    | 0.02872   | 4.23      | 0.4579    | 4.15      | 23.33     | 180.4     |
| 14        | 0.8535    | 0.1504    | 3.904     | 0.895     | 10.11     | 19.27     | 181.0     |
| 15        | 0.8321    | 0.05472   | 4.584     | 0.4853    | 14.54     | 17.26     | 314.6     |
| 16        | 0.8574    | 0.1117    | 2.571     | 0.684     | 7.299     | 3.064     | 272.5     |
| 17        | 0.3217    | 0.001     | 4.003     | 1.0       | 8.731     | 4.497     | 274.0     |
| 18        | 0.8482    | 0.1096    | 2.707     | 0.7031    | 11.17     | 22.82     | 78.15     |
| 19        | 0.8401

| 19        | 0.7895    | 0.1115    | 2.655     | 0.7233    | 7.353     | 3.148     | 272.6     |
| 20        | 0.8037    | 0.2       | 2.962     | 1.0       | 7.056     | 3.457     | 272.5     |
| 21        | 0.3158    | 0.002686  | 2.355     | 0.6387    | 2.712     | 16.91     | 229.1     |
| 22        | 0.7471    | 0.05254   | 4.857     | 0.2515    | 12.21     | 12.49     | 115.3     |
| 23        | 0.7449    | 0.06679   | 1.646     | 0.3375    | 9.999     | 10.33     | 488.5     |
| 24        | 0.7552    | 0.02503   | 2.029     | 0.2521    | 14.07     | 14.75     | 163.4     |
| 25        | 0.759     | 0.07754   | 3.498     | 0.5247    | 8.575     | 20.35     | 485.0     |
| 26        | 0.7765    | 0.08419   | 2.718     | 0.3036    | 10.01     | 23.18     | 78.11     |
| 27        | 0.8002    | 0.07339   | 2.71      | 0.7791    | 7.438     | 3.203     | 272.6     |
| 28        | 0.2566    | 0.001     | 2.671     | 0.738     | 7.254     | 3.164     | 272.5     |
| 29        | 0.7614

| 29        | 0.6554    | 0.0739    | 1.727     | 0.8856    | 9.309     | 16.39     | 458.7     |
| 30        | 0.7659    | 0.1083    | 2.952     | 0.9125    | 9.517     | 22.67     | 77.26     |
| 31        | 0.03275   | 0.001     | 3.039     | 0.7681    | 9.604     | 22.76     | 77.32     |
| 32        | 0.661     | 0.1639    | 1.364     | 0.2557    | 4.134     | 24.37     | 234.2     |
| 33        | 0.6629    | 0.1624    | 4.524     | 0.416     | 14.96     | 5.073     | 309.8     |
| 34        | 0.7544    | 0.08626   | 2.077     | 0.5619    | 4.392     | 15.74     | 353.1     |
| 35        | 0.6412    | 0.1779    | 1.378     | 0.1688    | 13.33     | 18.66     | 445.5     |
本轮结果: R²=0.9964, RMSE=0.0073

========== 迭代次数 5/10 ==========
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.8307    | 0.08399   | 3.881     | 0.1001

| 2         | 0.8668    | 0.03807   | 2.382     | 0.4571    | 9.005     | 11.64     | 345.8     |
| 3         | 0.8233    | 0.04169   | 4.512     | 0.1246    | 10.72     | 11.6      | 283.8     |
| 4         | 0.7789    | 0.02894   | 1.792     | 0.8207    | 14.59     | 9.209     | 349.2     |
| 5         | 0.7634    | 0.1754    | 4.578     | 0.1765    | 2.508     | 5.906     | 440.3     |
| 6         | 0.822     | 0.02057   | 2.684     | 0.9621    | 8.931     | 17.91     | 164.6     |
| 7         | 0.8397    | 0.1376    | 4.339     | 0.1165    | 11.75     | 24.74     | 376.6     |
| 8         | 0.8601    | 0.05681   | 4.157     | 0.1929    | 7.823     | 22.9      | 153.9     |
| 9         | 0.7802    | 0.05827   | 1.52      | 0.1174    | 10.82     | 6.867     | 140.1     |
| 10        | 0.7757    | 0.09882   | 1.213     | 0.6167    | 3.907     | 15.55     | 352.9     |
| 11        | 0.8522    | 0.02136   | 2.656     | 0.725     | 7.384     | 3.149     | 272.6     |
| 12        | 0.854 

| 12        | 0.8068    | 0.1331    | 3.06      | 0.9501    | 9.625     | 22.78     | 77.36     |
| 13        | 0.7875    | 0.02872   | 4.23      | 0.4579    | 4.15      | 23.33     | 180.4     |
| 14        | 0.8266    | 0.1504    | 3.904     | 0.895     | 10.11     | 19.27     | 181.0     |
| 15        | 0.7865    | 0.05472   | 4.584     | 0.4853    | 14.54     | 17.26     | 314.6     |
| 16        | 0.8058    | 0.08677   | 2.722     | 0.7904    | 7.45      | 3.214     | 272.7     |
| 17        | 0.8089    | 0.1777    | 2.789     | 0.6977    | 5.599     | 16.38     | 429.7     |
| 18        | 0.7966    | 0.04745   | 2.549     | 0.6179    | 7.277     | 3.042     | 272.5     |
| 19        | 0.7975    | 0.07282   | 3.565     | 0.1792    | 11.31     | 11.29     | 283.8     |
| 20        | 0.2755    | 0.001     | 2.593     | 0.7562    | 7.303     | 3.099     | 272.7     |
| 21        | 0.3129    | 0.002686  | 2.355     | 0.6387    | 2.712     | 16.91     | 229.1     |
| 22        | 0.7888

| 22        | 0.8177    | 0.05254   | 4.857     | 0.2515    | 12.21     | 12.49     | 115.3     |
| 23        | 0.7698    | 0.06679   | 1.646     | 0.3375    | 9.999     | 10.33     | 488.5     |
| 24        | 0.8271    | 0.02503   | 2.029     | 0.2521    | 14.07     | 14.75     | 163.4     |
| 25        | 0.8218    | 0.07754   | 3.498     | 0.5247    | 8.575     | 20.35     | 485.0     |
| 26        | 0.7795    | 0.0751    | 4.215     | 0.9389    | 2.239     | 18.28     | 100.3     |
| 27        | 0.8528    | 0.1218    | 4.151     | 0.1004    | 9.24      | 19.82     | 49.71     |
| 28        | 0.8584    | 0.03317   | 2.668     | 0.7372    | 7.397     | 3.161     | 272.6     |
| 29        | 0.7514    | 0.0739    | 1.727     | 0.8856    | 9.309     | 16.39     | 458.7     |
| 30        | 0.8156    | 0.198     | 2.431     | 1.0       | 7.081     | 2.912     | 273.3     |
| 31        | 0.8449    | 0.0582    | 2.615     | 0.7701    | 7.332     | 3.106     | 272.7     |
| 32        | 0.7691

| 32        | 0.7895    | 0.1639    | 1.364     | 0.2557    | 4.134     | 24.37     | 234.2     |
| 33        | 0.8119    | 0.1624    | 4.524     | 0.416     | 14.96     | 5.073     | 309.8     |
| 34        | 0.8686    | 0.1733    | 2.877     | 0.9129    | 7.12      | 3.332     | 272.7     |
| 35        | 0.8674    | 0.2       | 2.521     | 0.8402    | 7.469     | 3.919     | 272.7     |
本轮结果: R²=0.9084, RMSE=0.0556

========== 迭代次数 10/10 ==========
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.7811    | 0.08399   | 3.881     | 0.1001    | 5.93      | 5.375     | 55.25     |
| 2         | 0.7836    | 0.03807   | 2.382     | 0.4571    | 9.005     | 11.64     | 345.8     |
| 3         | 0.7454    | 0.04169   | 4.512     | 0.1246    | 10.72     | 11.6      | 283.8     |
| 4         | 0.6919    | 0.02894   | 1.792     | 0.820